In [1]:
#!git clone https://github.com/RobustBench/robustbench
%cd /workspace/robustbench/
!pip3 install . --upgrade

Cloning into 'robustbench'...
remote: Enumerating objects: 4825, done.
remote: Counting objects: 100% (375/375), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 4825 (delta 242), reused 210 (delta 175), pack-reused 4450 (from 3)
Receiving objects: 100% (4825/4825), 6.11 MiB | 7.37 MiB/s, done.
Resolving deltas: 100% (3137/3137), done.
/workspace/robustbench


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Processing /workspace/robustbench
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/fra31/auto-attack.git (to revision a39220048b3c9f2cca9a4d3a54604793c68eca7e) to /tmp/pip-install-y8jiu9ly/autoattack_c08229ea117f4c6aa02a8b24ef60d177
  Running command git clone --filter=blob:none --quiet https://github.com/fra31/auto-attack.git /tmp/pip-install-y8jiu9ly/autoattack_c08229ea117f4c6aa02a8b24ef60d177
  Running command git rev-parse -q --verify 'sha^a39220048b3c9f2cca9a4d3a54604793c68eca7e'
  Running command git fetch -q https://github.com/fra31/auto-attack.git a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Resolved https://github.com/fra31/auto-attack.git to commit a39220048b3c9f2cca9a4d3a54604793c68eca7e
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.1 

In [24]:
#!pip3 uninstall robustbench -y

Found existing installation: robustbench 1.1
Uninstalling robustbench-1.1:
  Successfully uninstalled robustbench-1.1


In [1]:
%cd /workspace/robustbench/
from robustbench import load_model
import torch
# Load a model from the model zoo
device = torch.device("cuda:0")
model = load_model(model_name='Peng2023Robust',
                   dataset='cifar10',
                   threat_model='Linf').to(device)
net = load_model(model_name='Peng2023Robust',
                   dataset='cifar10',
                   threat_model='Linf').to(device)
# Evaluate the Linf robustness of the model using AutoAttack
#from robustbench.eval import benchmark
#clean_acc, robust_acc = benchmark(model,
#                                  dataset='cifar10',
#                                  threat_model='Linf', device=device, eps=8/255)

/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/workspace/robustbench


In [2]:
!pip3 install torchattacks


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch
import torchvision
def set_seeds(seed):
    torch.manual_seed(seed)
    import random
    random.seed(seed)
    import numpy as np
    np.random.seed(seed)
    torch.mps.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic=True
transforms_train = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.RandomHorizontalFlip()])
transforms_test = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root="/cifar10/",
                               train=True,
                               transform=transforms_test,
                               download=True)
testset = torchvision.datasets.CIFAR10(root="/cifar10/",
                               train=False,
                               transform=transforms_test,
                               download=True)
set_seeds(0)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=1)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True, num_workers=1)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
criterion = torch.nn.CrossEntropyLoss()
import math
def evaluate_attacked_model(loader, phase, atk, limit=5000, model=model):
    correct = 0
    total = 0
    counter = 0
    total_loss = 0.0
    for data in loader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        #print(labels)
        model.train()
        images = atk(images, labels)
        model.eval()
        outputs = model(images.to(device))
        #print(outputs.shape)
        # the class with the highest energy is what we choose as prediction
        #print(outputs.data)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total >= limit:
            break
    print(f'{phase} accuracy: {100 * correct / total} ({correct} / {total}) %')
    return correct, total_loss / math.ceil(limit / 16)

In [13]:
import torchattacks
atk = torchattacks.PGD(model, steps=20)
# 20-step PGD: 761 / 1024
evaluate_attacked_model(testloader, "test", atk, limit=1000, model=net)

test accuracy: 74.31640625 (761 / 1024) %


(761, 0.44063784016503227)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


<All keys matched successfully>

In [6]:
import torchattacks
LR = 0.0001
import torch.optim as optim
import random
MODELTYPE = "Peng2023Robust_extratrain"
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 1)
pgd5 = torchattacks.PGD(net, steps=5)
pgd40 = torchattacks.PGD(net, steps=40)
aa = torchattacks.AutoAttack(net)
#pgd10.set_mode_targeted_least_likely(1)
vanila = torchattacks.VANILA(net)
record, record_fn = float("inf"), ""
for epoch in range(4):  # loop over the dataset multiple times
    # test the 625 / 125 spacing
    # to even out the intervals when plotted out as a graph
    print(scheduler.get_last_lr())
    running_loss = 0.0
    adv_loss, clean_loss, adv_runs = 0.0, 0.0, 0
    for i, data in enumerate(trainloader, 0):
        if i % 125 == 0:
            # sample 2000 data points from the train and test sets
            # try: borrow the batch mixing method from curriculum adversarial training and merge two batches into one big batch
            # will need to adjust the batch size to 8 though
            net.eval()
            #train_corrects, train_loss = evaluate_attacked_model(trainloader, "clean_train", vanila, limit=1000)
            #test_corrects, test_loss = evaluate_attacked_model(testloader, "clean_test", vanila, limit=1000)
            net.train()
            #print(train_corrects, train_loss)
            #print(test_corrects, test_loss)
            net.eval()
            #train_corrects, train_loss = evaluate_attacked_model(trainloader, "pgd5_train", pgd5, limit=200)
            #test_corrects, test_loss = evaluate_attacked_model(testloader, "pgd5_test", pgd5, limit=200)
            net.train()
            #print(train_corrects, train_loss)
            #print(test_corrects, test_loss)
            fn = f"models/{MODELTYPE}_lr{LR}_{epoch}epochs_{i}_batches.pth"
            torch.save(net.state_dict(), fn)
            net.eval()
            #train_corrects, train_loss = evaluate_attacked_model(testloader2, "aa_test", aa, limit=16)
            net.train()
            #print(test_corrects, test_loss)
        if epoch == 3:
            break # stop training here, the last "epoch" is for reporting
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        adv = random.randint(1, 1)
        if adv == 1:
            inputs = pgd5(inputs, labels)
        outputs = net(inputs) # can we actually get rid of the clean samples completely?
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        #print(loss.item())
        if i % 25 == 24:    # print every 125 mini-batches
            print(f'[{epoch}, {i + 1:5d}] loss: {running_loss / 25}')
            #print(f"Adv loss: {adv_loss / adv_runs})")
            running_loss = 0.0
            adv_loss, clean_loss, adv_runs = 0.0, 0.0, 0
            #test_corrects, test_loss = evaluate_attacked_model(testloader, "pgd10_test", pgd10, limit=200, model=net)
            with torch.no_grad():
                counter = 0
        if i % 625 == 624:
            scheduler.step()
# best clean / adv splits got with cosine (0.05 -> 0.005): 93.75% / 41.96%
# try to mix and match it with SAM (sharpness aware minimization) (doubles the training time though for standard thing)
# and also with target logit loss
# if cosine layer wins the key might be less overfitting (it's harder to overfit on cosine similarity)
# because the decision boundaries are more well-rounded?
# quickly pull up some t-SNE (scikit-learn) plots of feature maps to look at the feature maps of the linear vs cosine layers
# best clean / adv splits got with linear (0.02? -> 0.001): 91.01% / 28.57% (?)

[0.0001]
[0,    25] loss: 0.44610306739807126
[0,    50] loss: 0.4000282430648804
[0,    75] loss: 0.370904735326767
[0,   100] loss: 0.3372477114200592
[0,   125] loss: 0.3378241276741028
[0,   150] loss: 0.3339101159572601
[0,   175] loss: 0.3083159124851227
[0,   200] loss: 0.2938941514492035
[0,   225] loss: 0.2775013089179993
[0,   250] loss: 0.2770329761505127
[0,   275] loss: 0.26669916629791257
[0,   300] loss: 0.2691108500957489
[0,   325] loss: 0.25969280779361725
[0,   350] loss: 0.26386243760585787
[0,   375] loss: 0.25311654806137085
[0,   400] loss: 0.25883460104465483
[0,   425] loss: 0.23149783432483673
[0,   450] loss: 0.20211881995201111
[0,   475] loss: 0.23929192304611205
[0,   500] loss: 0.21034965813159942
[0,   525] loss: 0.2344609558582306
[0,   550] loss: 0.24165017634630204
[0,   575] loss: 0.2013562110066414
[0,   600] loss: 0.20244475394487382
[0,   625] loss: 0.21512236326932907
[0,   650] loss: 0.18775148451328277
[0,   675] loss: 0.21841496855020523
[0,  

KeyboardInterrupt: 

In [13]:
net.load_state_dict(torch.load("/workspace/robustbench/models/Peng2023Robust_extratrain_lr0.0001_0epochs_625_batches.pth", weights_only=True))

<All keys matched successfully>

In [7]:
torch.save(net.state_dict(), "net_state_dict.pt")

In [16]:
import torchattacks
pgd10 = torchattacks.PGD(net)
test_corrects, test_loss = evaluate_attacked_model(testloader, "pgd10_test", pgd10, limit=10000)

pgd10_test accuracy: 75.94 (7594 / 10000) %


In [12]:
import torchattacks
pgd10 = torchattacks.PGD(model, steps=10)
# 75.x% robustness
test_corrects, test_loss = evaluate_attacked_model(testloader, "pgd10_test", pgd10, limit=10000, model=model)
# baseline: PGD-10: 75.05%; PGD-20: 74.33%

pgd10_test accuracy: 75.05 (7505 / 10000) %


In [14]:
import torchattacks
pgd10 = torchattacks.PGD(model, steps=10)
# managed to capture it before gradient obfuscation... maybe is low learning rate the key?
test_corrects, test_loss = evaluate_attacked_model(testloader, "pgd10_test", pgd10, limit=10000, model=net)
# got 75.35% pgd10_test transfer accuracy on 625-step training!!!

pgd10_test accuracy: 75.35 (7535 / 10000) %


In [40]:
atk = torchattacks.PGD(net, steps=20)
test_corrects, test_loss = evaluate_attacked_model(testloader, "atk_test", atk, limit=1000)

atk_test accuracy: 79.00390625 (809 / 1024) %
